# Class for the Dataset

In [4]:
import torch
from torch.utils.data import Dataset

## Defining my own dataset class, provide more freedom to design the problem 

In [8]:

class Dataset_ABC(Dataset):
    def __init__(self,
                 data,
                 input_bars_number = 8, # By default
                 target_bars_numbers = 8, # By default
                 bos_id=2, # Start Token
                 eos_id=3, # End Token
                 is_test=False):
        
        
        self.notes = []
        self.keys = []
        
        for (key,notes) in data:
            if notes == None:
                continue
 
            self.keys.append(key)
            self.notes.append(notes)
        
        self.context_bars_num = context_bars_num
        self.target_bars_num = target_bars_num
        self.bos_id = bos_id
        self.eos_id = eos_id
        self.is_test = is_test
        
        
        
        
    def __len__(self):
        return len(self.keys)
    
    def __getitem__(self,idx):
        
        notes = self.notes[idx]
        keys = self.keys[idx]
        
        if (not self.is_test):
            split_indx = self.context_bars_num

            # split notes to input and target
            context_notes = notes[split_indx - self.context_bars_num : split_indx]
            target_notes = notes[split_indx: split_indx + self.target_bars_num]
        else:
            context_notes = notes
            target_notes = []
            
        # Tokenization
        context_tokens = [self.bos_id] + keys #Start of the token
        target_tokens = [self.bos_id]

        for bar in context_notes:
            context_tokens += bar

        for bar in target_notes:
            target_tokens += bar

        context_tokens += [self.eos_id] # End of the token
        target_tokens += [self.eos_id]

        # Convert to tensor to be passed to the transformer
        context_tokens = torch.tensor(context_tokens, dtype=torch.long)
        target_tokens = torch.tensor(target_tokens, dtype=torch.long)

        return {"features": context_tokens, "target": target_tokens}
        
    